In [2]:
from matplotlib import transforms
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np

# Load the CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

# build model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

# training the model
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

# evaulate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')



100%|██████████| 170M/170M [00:47<00:00, 3.57MB/s] 


[Epoch 1, Batch 2000] loss: 2.204
[Epoch 1, Batch 4000] loss: 1.820
[Epoch 1, Batch 6000] loss: 1.670
[Epoch 1, Batch 8000] loss: 1.581
[Epoch 1, Batch 10000] loss: 1.536
[Epoch 1, Batch 12000] loss: 1.467
[Epoch 2, Batch 2000] loss: 1.412
[Epoch 2, Batch 4000] loss: 1.392
[Epoch 2, Batch 6000] loss: 1.358
[Epoch 2, Batch 8000] loss: 1.319
[Epoch 2, Batch 10000] loss: 1.316
[Epoch 2, Batch 12000] loss: 1.262
[Epoch 3, Batch 2000] loss: 1.227
[Epoch 3, Batch 4000] loss: 1.216
[Epoch 3, Batch 6000] loss: 1.228
[Epoch 3, Batch 8000] loss: 1.189
[Epoch 3, Batch 10000] loss: 1.196
[Epoch 3, Batch 12000] loss: 1.181
[Epoch 4, Batch 2000] loss: 1.115
[Epoch 4, Batch 4000] loss: 1.110
[Epoch 4, Batch 6000] loss: 1.116
[Epoch 4, Batch 8000] loss: 1.127
[Epoch 4, Batch 10000] loss: 1.105
[Epoch 4, Batch 12000] loss: 1.118
[Epoch 5, Batch 2000] loss: 1.033
[Epoch 5, Batch 4000] loss: 1.059
[Epoch 5, Batch 6000] loss: 1.050
[Epoch 5, Batch 8000] loss: 1.041
[Epoch 5, Batch 10000] loss: 1.036
[Epoc

In [5]:
# save the model
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [7]:
# load the model
net = Net()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>